# Задание 1

Импортируйте библиотеки pandas и numpy.
Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных.
Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки
составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42.
Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.
Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.
Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

%matplotlib inline

In [2]:
boston = load_boston()

In [3]:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [4]:
data = boston.data # Данные о недвижимости 
feature_names = boston.feature_names # Название признаков
target = boston.target # Цена на недвижимость

In [5]:
# Создадим датафрейм X и y
X = pd.DataFrame(data, columns=feature_names)
y = pd.DataFrame(target, columns=['Price'])

In [6]:
# Разобьем датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
# Создаем модель линейной регрессии
lr = LinearRegression()

In [8]:
# Обучем модель на тренировочных данных
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [9]:
# Спрогнозируем цены y_pred на тестовых данных X_test
y_pred_reg = lr.predict(X_test)

In [10]:
# Проведем визуальную оценку предсказанных данных
check_test_reg = pd.DataFrame({'y_test':y_test['Price'],
                          'y_pred_reg':y_pred_reg.flatten()}, columns=['y_test', 'y_pred_reg'])

In [11]:
check_test_reg.head()

,y_test,y_pred_reg
173,23.6,28.648960
274,32.4,36.495014
491,13.6,15.411193
72,22.8,25.403213
452,16.1,18.855280


In [12]:
R2_reg = r2_score(y_test['Price'], y_pred_reg.flatten())
R2_reg

0.711226005748496

# Задание 2

Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble.
Сделайте агрумент n_estimators равным 1000,
max_depth должен быть равен 12 и random_state сделайте равным 42.
Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression,
но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0],
чтобы получить из датафрейма одномерный массив Numpy,
так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.
Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
Напишите в комментариях к коду, какая модель в данном случае работает лучше.

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [14]:
# Создаем модель с использованием RandomForestRegressor
model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

In [15]:
# Обучим модель на тренировочных данных
model.fit(X_train, y_train.values[:,0])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
# Сделайте предсказание на тестовых данных
y_pred_tree = model.predict(X_test)

In [17]:
# Проведем визуальную оценку предсказанных данных
check_test_tree = pd.DataFrame({'y_test':y_test['Price'],
                          'y_pred_tree':y_pred_tree}, columns=['y_test', 'y_pred_tree'])
check_test_tree.head()

,y_test,y_pred_tree
173,23.6,22.846138
274,32.4,31.156114
491,13.6,16.297226
72,22.8,23.821036
452,16.1,17.212148


In [18]:
R2_tree = r2_score(y_test['Price'], y_pred_tree)
R2_tree

0.8749965273218174

В данном случае, по метрике R^2, модель постороенная с помощью RandomForestRegressor работает лучше, чем LinearRegression, так как R2_tree > R2_reg

# Задание 3

Вызовите документацию для класса RandomForestRegressor,
найдите информацию об атрибуте feature_importances_.
С помощью этого атрибута найдите сумму всех показателей важности,
установите, какие два признака показывают наибольшую важность.

In [19]:
?RandomForestRegressor

In [20]:
# Определим сумму всех показателей важности
print(f'Сумма всех показателей важности: {sum(model.feature_importances_):.4}')

Сумма всех показателей важности: 1.0


In [21]:
# Определим два наиболее значимых признака 
num_largest = 2
indices = (-model.feature_importances_).argpartition(num_largest, axis=None)[:num_largest]
print(f'Наиболее значимые два признака: {feature_names[indices][0]}, {feature_names[indices][1]}')

Наиболее значимые два признака: LSTAT, RM
